In [ ]:
library(tableone)
library(ggsurvfit)
library(boot)
library(dplyr)
library(sandwich)
library(cobalt)
library(speedglm)
library(data.table)
library(splitstackshape)

# Import data

In [ ]:
df = read.csv('/home/jovyan/work/RALES TRIAL/1A/masterfiles/masterfile_long_format.csv')

In [ ]:
names(df)[names(df) == 'client_idcode'] <- 'id'
options(scipen=999) # disable printing results in scientific notation 
K <-17 # set time point for estimation of risks

In [ ]:
head(df)

In [ ]:
nrow(df)

# Descriptive table 

In [ ]:
# List of variables to be included in the table
myVars <- c("age", "gender","creatinine", "lvef", "potassium", "ace_inhibitors", "ethnicity")

# List of categorical variables
catVars <- c("gender", "ace_inhibitors", "ethnicity")

# List of continuous variables which should be displayed as median (IQR)
medVars <- c("age","creatinine", "lvef", "potassium")

# Create table 1
tab1 <- CreateTableOne(vars = myVars, # set descriptive variables
                       strata = "trt", # define stratifying variable
                       data = df[which(df$time==0),], # baseline
                       factorVars = catVars) # define categorical variables

In [ ]:
print(tab1,
      nonnormal = medVars,
      formatOptions = list(big.mark = ","),
      test = FALSE)

# KM check

In [ ]:
fit.kim <- survfit(Surv(tstart,tstop,death) ~ trt,
                   conf.type='log-log',
                   data=df)

summary(fit.kim, times = K)

# Pooled logistic regression (without confounders & excluding any product terms between treatment groups indicator and follow-up time)

In [ ]:
fit.pool <- glm(formula = death==1 ~ trt + time + timesqr,
                 family = binomial(link = 'logit'),
                 data = df)
options(warn=0)
summary(fit.pool)

In [ ]:
# Exponentiate coefficient for trt - this should give you approx the same result as the Cox Proportional Hazards Model

round(exp(summary(fit.pool)$coef["trt","Estimate"]), 4)

# Pooled logistic regression (without confounders) 

In [ ]:
# Include product terms between time and treatment
options(warn=-1) # Suppress warning messages
fit.pool <- glm(formula = death==1 ~ trt + time + timesqr +
                  I(trt*time) +
                  I(trt*timesqr),
                family = binomial(link = 'logit'),
                data = df)
options(warn=-0)
# Note that this model will take a few seconds to run

# Print results
summary(fit.pool)

### Transform estimates to risks at each time point in each arm

In [ ]:
# Create a dataset to store results
# Include all time points under each treatment level
trt0 <- data.frame(cbind(seq(0, K-1),0,(seq(0, K-1))^2))
trt1 <- data.frame(cbind(seq(0, K-1),1,(seq(0, K-1))^2))

In [ ]:
# Set column names
colnames(trt0) <- c("time", "trt", "timesqr")
colnames(trt1) <- c("time", "trt", "timesqr")

In [ ]:
# Extract predicted values from pooled logistic regression model
# Predicted values correspond to discrete-time hazards
trt0$p.event0 <- predict(fit.pool, trt0, type="response")
trt1$p.event1 <- predict(fit.pool, trt1, type="response")

In [ ]:
# Estimate survival probabilities from hazards
# S(t) = cumulative product of (1 - h(t))
trt0$surv0 <- cumprod(1 - trt0$p.event0)
trt1$surv1 <- cumprod(1 - trt1$p.event1)

In [ ]:
# Estimate risks from survival probabilities
# Risk = 1 - S(t)
trt0$risk0 <- 1 - trt0$surv0
trt1$risk1 <- 1 - trt1$surv1

In [ ]:
# Prepare data
graph.pred <- merge(trt0, trt1, by=c("time", "timesqr"))
# Edit data frame to reflect that risks are estimated at the END of each interval
graph.pred$time_0 <- graph.pred$time + 1
zero <- data.frame(cbind(0,0,0,0,1,0,1,0,1,0,0))
zero <- setNames(zero,names(graph.pred))
graph <- rbind(zero, graph.pred)

### Use pooled logistic regression estimates to compute causal estimates

In [ ]:
# 2-year risk in the non spiro group
risk0.plr <- graph$risk0[which(graph$time==K-1)]
round(risk0.plr, 5)

In [ ]:
# 2-year risk in the spiro group
risk1.plr <- graph$risk1[which(graph$time==K-1)]
round(risk1.plr, 5)

In [ ]:
# 2-year risk difference
rd.plr <- risk1.plr - risk0.plr
round(rd.plr, 4)

In [ ]:
# 2-year risk ratio
rr.plr <- risk1.plr / risk0.plr
round(rr.plr, 2)

### Obtain percentile-based bootstrapped 95% CIs for each quantity

In [ ]:
# Create a function to obtain risks, RD, and RR from each bootstrap sample
risk.boot <- function(data, indices) {
  # Select individuals into each bootstrapped sample
  ids <- unique(data$id)
  boot.ids <- data.frame(id = ids[indices])
 
  # Subset person-time data to individuals selected into the bootstrapped sample
  d <- left_join(boot.ids, data, by = "id", relationship = "many-to-many")
 
  # Fit pooled logistic model to estimate discrete hazards
  options(warn=-1)
  pool.boot <- glm(formula = death==1 ~ trt + time + timesqr +
                     I(trt*time) +
                     I(trt*timesqr),
                   family = binomial(link = 'logit'),
                   data = d)
  options(warn=0)
 
  # Create a dataset to store results
  # Include all time points under each treatment level
  trt0 <- data.frame(cbind(seq(0, K-1),0,(seq(0, K-1))^2))
  trt1 <- data.frame(cbind(seq(0, K-1),1,(seq(0, K-1))^2))
 
  # Set column names
  colnames(trt0) <- c("time", "trt", "timesqr")
  colnames(trt1) <- c("time", "trt", "timesqr")
 
  # Extract predicted values from pooled logistic regression model
  # Predicted values correspond to discrete-time hazards
  trt0$p.event0 <- predict(pool.boot, trt0, type="response")
  trt1$p.event1 <- predict(pool.boot, trt1, type="response")
 
  # Convert from discrete-time hazards to survival probabilities
  # S(t) = cumulative product of (1 - h(t))
  trt0$surv0 <- cumprod(1 - trt0$p.event0)
  trt1$surv1 <- cumprod(1 - trt1$p.event1)
 
  # Convert from survival to risks
  # Risk = 1 - S(t)
  trt0$risk0 <- 1 - trt0$surv0
  trt1$risk1 <- 1 - trt1$surv1
 
  # Merge data from two arms and format
  graph <- merge(trt0, trt1, by=c("time", "timesqr"))
  graph$rd <- graph$risk1-graph$risk0
  graph$rr <- graph$risk1/graph$risk0
  return(c(graph$risk0[which(graph$time==K-1)],
           graph$risk1[which(graph$time==K-1)],
           graph$rd[which(graph$time==K-1)],
           graph$rr[which(graph$time==K-1)]))
}

In [ ]:
# Run 2,000 bootstrap samples 
set.seed(865)
risk.results <- boot(data = df,
                     statistic = risk.boot,
                     R=2000)

In [ ]:
# 95% CI for risk ratio
boot.ci(risk.results,
        conf = 0.95,
        type = "perc",
        index = 4) # create CI for fourth statistic (rr) returned by boot()

# Pooled logistic regression (adjusting for confounders using IPTW) 

In [ ]:
# Estimation of propensity scores with a logistic model
ps.treatment <- glm(formula = trt==1 ~  lvef + creatinine + potassium + age + as.factor(gender) + as.factor(ace_inhibitors),
                family = binomial(link = 'logit'),
                data = df[which(df$time==0),])

summary(ps.treatment)

In [ ]:
#  Estimate propensity scores
df$ps_treatment <- predict(ps.treatment, df, type = "response")

In [ ]:
head(df, n=20)

In [ ]:
summary(df$ps_treatment[df$trt==1])
summary(df$ps_treatment[df$trt==0])

### Verifying the Accuracy and Reliability of Propensity Score Calculation

In [ ]:
subset_df <- df[df$time == 0, ]

In [ ]:
head(subset_df)

In [ ]:
# Estimation of propensity scores with a logistic model
ps.treatment <- glm(formula = trt==1 ~  lvef + creatinine + potassium + age + as.factor(gender) + as.factor(ace_inhibitors),
                family = binomial(link = 'logit'),
                data = subset_df)

summary(ps.treatment)

In [ ]:
#  Estimate propensity scores
subset_df$ps_treatment_2 <- predict(ps.treatment, subset_df, type = "response")

In [ ]:
head(subset_df)

### Estimate the inverse probability weights 

In [ ]:
df$w_a <- ifelse(df$trt==1,
                 1/df$ps_treatment,
                 1/(1-df$ps_treatment))

In [ ]:
# Min, 25th percentile, median, mean, SD, 75th percentile, and max 
summary(df$w_a)
sd(df$w_a)

In [ ]:
# Create the histogram
hist(df[which(df$time==0),]$w_a, main = "IPTW Weights Distribution", xlab = "IPTW Weights")

# Calculate percentiles (you can change the percentiles as needed)
percentiles <- quantile(df[which(df$time==0),]$w_a, probs = c(0.95, 0.99))

# Add vertical lines for the percentiles
abline(v = percentiles, col = "red", lty = 2)

# Add a legend to indicate the percentiles
legend("topright", legend = paste0("P", c(95, 99), " = ", percentiles), col = "red", lty = 2, bty = "n")

In [ ]:
head(df, n=20)

###  Assessing the balance of covariates between treatment groups 

In [ ]:
subset_df <- df[df$time == 0, ]

covariates <- subset(subset_df, select = c("age", "gender","creatinine", "lvef", "potassium", "ace_inhibitors"))
bal.tab(covariates, treat =subset_df$trt, weights=subset_df$w_a,un=TRUE)

###  Fit pooled logistic regression with weights 

In [ ]:
# Include product terms between time and treatment
options(warn=-1) # Suppress warning messages
fit.pool <- speedglm(formula = death==1 ~ trt + time + timesqr +
                                    I(trt*time) +
                                    I(trt*timesqr),
                family = binomial(link = 'logit'),
                data = df,
               weights=df$w_a)
options(warn=-0)
# Note that this model will take a few seconds to run

# Print results
summary(fit.pool)

### Transform estimates to risks at each time point in each arm

In [ ]:
# Create a dataset to store results
# Include all time points under each treatment level
trt0 <- data.frame(cbind(seq(0, K-1),0,(seq(0, K-1))^2))
trt1 <- data.frame(cbind(seq(0, K-1),1,(seq(0, K-1))^2))

In [ ]:
# Set column names
colnames(trt0) <- c("time", "trt", "timesqr")
colnames(trt1) <- c("time", "trt", "timesqr")

In [ ]:
# Extract predicted values from pooled logistic regression model
# Predicted values correspond to discrete-time hazards
trt0$p.event0 <- predict(fit.pool, trt0, type="response")
trt1$p.event1 <- predict(fit.pool, trt1, type="response")

In [ ]:
# Estimate survival probabilities from hazards
# S(t) = cumulative product of (1 - h(t))
trt0$surv0 <- cumprod(1 - trt0$p.event0)
trt1$surv1 <- cumprod(1 - trt1$p.event1)

In [ ]:
# Estimate risks from survival probabilities
# Risk = 1 - S(t)
trt0$risk0 <- 1 - trt0$surv0
trt1$risk1 <- 1 - trt1$surv1

In [ ]:
# Prepare data
graph.pred <- merge(trt0, trt1, by=c("time", "timesqr"))
# Edit data frame to reflect that risks are estimated at the END of each interval
graph.pred$time_0 <- graph.pred$time + 1
zero <- data.frame(cbind(0,0,0,0,1,0,1,0,1,0,0))
zero <- setNames(zero,names(graph.pred))
graph <- rbind(zero, graph.pred)

### Use pooled logistic regression estimates to compute causal estimates

In [ ]:
# 2-year risk in the non spiro group

risk0.plr <- graph$risk0[which(graph$time==K-1)]
round(risk0.plr, 5)

In [ ]:
# 2-year risk in the  spiro group

risk1.plr <- graph$risk1[which(graph$time==K-1)]
round(risk1.plr, 5)

In [ ]:
# 2-year risk difference

rd.plr <- risk1.plr - risk0.plr
round(rd.plr, 4)


In [ ]:
# 2-year risk ratio

rr.plr <- risk1.plr / risk0.plr
round(rr.plr, 2)

### Obtain percentile-based bootstrapped 95% CIs for each quantity

In [ ]:
risk.boot <- function(data, indices) {
  # Select individuals into each bootstrapped sample
  ids <- unique(data$id)
  boot.ids <- data.frame(id = ids[indices])
  
  # Subset person-time data to individuals selected into the bootstrapped sample
  d <- left_join(boot.ids, data, by = "id", relationship = "many-to-many")  
    
  # Calculate weights based on propensity scores
  ps.treatment <- glm(formula = trt == 1 ~ lvef + creatinine + potassium + age + as.factor(gender) + as.factor(ace_inhibitors),
                      family = binomial(link = 'logit'),
                      data = d[which(d$time==0),])
  d$ps_treatment <- predict(ps.treatment, d, type = "response")
  
  d$w_a <- ifelse(d$trt == 1,
                   1 / d$ps_treatment,
                   1 / (1 - d$ps_treatment))
  
  # Fit pooled logistic model to estimate discrete hazards with weights
  options(warn=-1)
  pool.boot <- glm(formula = death == 1 ~ trt + time + timesqr +
                     I(trt*time) +
                     I(trt*timesqr),
                   family = binomial(link = 'logit'),
                   data = d,
                   weights = d$w_a)  # Use the weights here
  options(warn=0)
  
  # Create a dataset to store results
  # Include all time points under each treatment level
  trt0 <- data.frame(cbind(seq(0, K-1), 0, (seq(0, K-1))^2))
  trt1 <- data.frame(cbind(seq(0, K-1), 1, (seq(0, K-1))^2))
  
  # Set column names
  colnames(trt0) <- c("time", "trt", "timesqr")
  colnames(trt1) <- c("time", "trt", "timesqr")
  
  # Extract predicted values from pooled logistic regression model
  # Predicted values correspond to discrete-time hazards
  trt0$p.event0 <- predict(pool.boot, trt0, type="response")
  trt1$p.event1 <- predict(pool.boot, trt1, type="response")
  
  # Convert from discrete-time hazards to survival probabilities
  # S(t) = cumulative product of (1 - h(t))
  trt0$surv0 <- cumprod(1 - trt0$p.event0)
  trt1$surv1 <- cumprod(1 - trt1$p.event1)
  
  # Convert from survival to risks
  # Risk = 1 - S(t)
  trt0$risk0 <- 1 - trt0$surv0
  trt1$risk1 <- 1 - trt1$surv1
  
  # Merge data from two arms and format
  graph <- merge(trt0, trt1, by=c("time", "timesqr"))
  graph$rd <- graph$risk1 - graph$risk0
  graph$rr <- graph$risk1 / graph$risk0
  return(c(graph$risk0[which(graph$time==K-1)],
           graph$risk1[which(graph$time==K-1)],
           graph$rd[which(graph$time==K-1)],
           graph$rr[which(graph$time==K-1)]))
}

In [ ]:
# Run 2000 bootstrap samples 
set.seed(865)
risk.results <- boot(data = df,
                     statistic = risk.boot,
                     R=2000)

In [ ]:
# 95% CI for risk ratio
boot.ci(risk.results,
        conf = 0.95,
        type = "perc",
        index = 4) # create CI for fourth statistic (rr) returned by boot()